In [8]:
import pandas as pd
import numpy as np
import re
from string import ascii_letters, digits
import utils.cleaning_dicts
import os
import sys

In [9]:
print(utils.cleaning_dicts.clean_team_pff)

{'arz': 'ari', 'blt': 'bal', 'clv': 'cle', 'hst': 'hou', 'la': 'lar', 'sd': 'lac', 'sl': 'lar'}


In [10]:
os.chdir(sys.path[0])

In [11]:
!ls

chromedriver			   historic_data  other_data  rt.csv
create_modeling_data_sample.ipynb  modeling_data  pfr	      spreads_data
current_data			   notebooks	  README.md   utils


### Read in and process all pff position datasets

In [12]:
####################################################################################
                ###   Read-in and clean all passing datasets ###
####################################################################################

passing_depth = pd.read_csv('./historic_data/pff_data/passing_depth_hist.csv')
passing_allowed_pressure = pd.read_csv('./historic_data/pff_data/passing_allowed_pressure_hist.csv')
passing_pressure = pd.read_csv('./historic_data/pff_data/passing_pressure_hist.csv')
passing_concept = pd.read_csv('./historic_data/pff_data/passing_concept_hist.csv')
time_in_pocket = pd.read_csv('./historic_data/pff_data/time_in_pocket_hist.csv')
passing_summ_conc = pd.read_csv('./historic_data/pff_data/passing_summ_conc_hist.csv')


def drop_non_qbs(df):
    df=df.rename(columns={"player_id": "numeric_id"})
    df=df[df['position'] == 'QB']
    df['position']=df['position'].astype(str).str.lower()
    df['team_name']=df['team_name'].astype(str).str.lower()       
    df['player']=df['player'].str.replace('[^a-zA-Z0-9]', '').str.lower()
    df['team_name']=df['team_name'].str.lower()
    df['year'] = df['year'].astype(str)
    df['week'] = df['week'].astype(str)
    
        ##  pass team name through dictionary to clean ##
    df['team_name'] = df['team_name'].map(utils.cleaning_dicts.clean_team_pff).fillna(df['team_name'])
    df['position'] = df['position'].map(utils.cleaning_dicts.pos_dict).fillna(df['position'])

    
    df.insert(0, "p_id", (df['player']+'_'+df['team_name']+'_'+df['year']+'_'+df['week']))
    df.insert(1, "unique_team_id", (df['team_name']+'_'+df['year']+'_'+df['week']))
    df.insert(2, "player_team_id", (df['player']+'_'+df['team_name']+'_'+df['year']))
    df.insert(3, "team_id_impute", (df['team_name']+'_'+df['year']))
    return df
    
passing_depth = drop_non_qbs(passing_depth)
passing_allowed_pressure = drop_non_qbs(passing_allowed_pressure)
passing_pressure = drop_non_qbs(passing_pressure)
passing_concept = drop_non_qbs(passing_concept)
time_in_pocket = drop_non_qbs(time_in_pocket)
passing_summ_conc = drop_non_qbs(passing_summ_conc)


passing_depth = passing_depth[passing_depth.columns.drop(list(passing_depth.filter(regex='left|right|center')))]

####################################################################################
				###   Read-in and clean all receiving datasets ###
####################################################################################

rec_summ_conc = pd.read_csv('./historic_data/pff_data/rec_summ_conc_hist.csv')
receiving_concept = pd.read_csv('./historic_data/pff_data/receiving_concept_hist.csv')
receiving_depth = pd.read_csv('./historic_data/pff_data/receiving_depth_hist.csv')
receiving_scheme = pd.read_csv('./historic_data/pff_data/receiving_scheme_hist.csv')

def drop_non_recs(df):
    df=df.rename(columns={"player_id": "numeric_id"})
    df= df[df.position.str.match('WR|TE|HB|FB')]
    df['position']=df['position'].astype(str).str.lower()
    df['team_name']=df['team_name'].astype(str).str.lower()       
    df['player']=df['player'].str.replace('[^a-zA-Z0-9]', '').str.lower()
    df['team_name']=df['team_name'].str.lower()
    df['year'] = df['year'].astype(str)
    df['week'] = df['week'].astype(str)
    
        ##  pass team name through dictionary to clean ##
    df['team_name'] = df['team_name'].map(utils.cleaning_dicts.clean_team_pff).fillna(df['team_name'])
    df['position'] = df['position'].map(utils.cleaning_dicts.pos_dict).fillna(df['position'])

    
    df.insert(0, "p_id", (df['player']+'_'+df['team_name']+'_'+df['year']+'_'+df['week']))
    df.insert(1, "unique_team_id", (df['team_name']+'_'+df['year']+'_'+df['week']))
    df.insert(2, "player_team_id", (df['player']+'_'+df['team_name']+'_'+df['year']))
    df.insert(3, "team_id_impute", (df['team_name']+'_'+df['year']))
    return df

rec_summ_conc = drop_non_recs(rec_summ_conc)
receiving_concept = drop_non_recs(receiving_concept)
receiving_depth = drop_non_recs(receiving_depth)
receiving_scheme = drop_non_recs(receiving_scheme)


####################################################################################
				###   Read-in and clean all rushing datasets ###
####################################################################################

rush_summ_conc = pd.read_csv('./historic_data/pff_data/rush_summ_conc_hist.csv')

def drop_non_rbs(df):
    df=df.rename(columns={"player_id": "numeric_id"})
    df= df[df.position.str.match('WR|HB|FB')]
    df['position']=df['position'].astype(str).str.lower()
    df['team_name']=df['team_name'].astype(str).str.lower()       
    df['player']=df['player'].str.replace('[^a-zA-Z0-9]', '').str.lower()
    df['team_name']=df['team_name'].str.lower()
    df['year'] = df['year'].astype(str)
    df['week'] = df['week'].astype(str)
        ##  pass team name through dictionary to clean ##
    df['team_name'] = df['team_name'].map(utils.cleaning_dicts.clean_team_pff).fillna(df['team_name'])
    df['position'] = df['position'].map(utils.cleaning_dicts.pos_dict).fillna(df['position'])

    
    df.insert(0, "p_id", (df['player']+'_'+df['team_name']+'_'+df['year']+'_'+df['week']))
    df.insert(1, "unique_team_id", (df['team_name']+'_'+df['year']+'_'+df['week']))
    df.insert(2, "player_team_id", (df['player']+'_'+df['team_name']+'_'+df['year']))
    df.insert(3, "team_id_impute", (df['team_name']+'_'+df['year']))
    return df

rush_summ_conc = drop_non_rbs(rush_summ_conc)


####################################################################################
				###   Read-in and clean all blocking datasets ###
####################################################################################


block_summ_conc = pd.read_csv('./historic_data/pff_data/block_summ_conc_hist.csv')
offense_pass_blocking = pd.read_csv('./historic_data/pff_data/offense_pass_blocking_hist.csv')
offense_run_blocking = pd.read_csv('./historic_data/pff_data/offense_run_blocking_hist.csv')


def drop_non_ols(df):
    df=df.rename(columns={"player_id": "numeric_id"})
    df = df[df['position'].notna()]
    df= df[df.position.str.match('T|C|G|TE')]
    df['position']=df['position'].astype(str).str.lower()
    df['team_name']=df['team_name'].astype(str).str.lower()       
    df['player']=df['player'].str.replace('[^a-zA-Z0-9]', '').str.lower()
    df['team_name']=df['team_name'].str.lower()
    df['year'] = df['year'].astype(str)
    df['week'] = df['week'].astype(str)
        ##  pass team name through dictionary to clean ##
    df['team_name'] = df['team_name'].map(utils.cleaning_dicts.clean_team_pff).fillna(df['team_name'])
    df['position'] = df['position'].map(utils.cleaning_dicts.pos_dict).fillna(df['position'])

    
    df.insert(0, "p_id", (df['player']+'_'+df['team_name']+'_'+df['year']+'_'+df['week']))
    df.insert(1, "unique_team_id", (df['team_name']+'_'+df['year']+'_'+df['week']))
    df.insert(2, "player_team_id", (df['player']+'_'+df['team_name']+'_'+df['year']))
    df.insert(3, "team_id_impute", (df['team_name']+'_'+df['year']))
    return df


block_summ_conc	= drop_non_ols(block_summ_conc)
offense_pass_blocking = drop_non_ols(offense_pass_blocking)
offense_run_blocking = drop_non_ols(offense_run_blocking)



####################################################################################
				###   Read-in and clean all defensive datasets ###
####################################################################################

def_summ_conc = pd.read_csv('./historic_data/pff_data/def_summ_conc_hist.csv')
pass_rush_summary = pd.read_csv('./historic_data/pff_data/pass_rush_summary_hist.csv')
run_defense_summary = pd.read_csv('./historic_data/pff_data/run_defense_summary_hist.csv')
defense_coverage_scheme = pd.read_csv('./historic_data/pff_data/defense_coverage_scheme_hist.csv')
defense_coverage_summary = pd.read_csv('./historic_data/pff_data/defense_coverage_summary_hist.csv')
slot_coverage = pd.read_csv('./historic_data/pff_data/slot_coverage_hist.csv')


def drop_non_def(df):
    df=df.rename(columns={"player_id": "numeric_id"})
    df['position']=df['position'].astype(str).str.lower()
    df['team_name']=df['team_name'].astype(str).str.lower()       
    df['player']=df['player'].str.replace('[^a-zA-Z0-9]', '').str.lower()
    df['team_name']=df['team_name'].str.lower()
    df['year'] = df['year'].astype(str)
    df['week'] = df['week'].astype(str)
        ##  pass team name through dictionary to clean ##
    df['team_name'] = df['team_name'].map(utils.cleaning_dicts.clean_team_pff).fillna(df['team_name'])
    df['position'] = df['position'].map(utils.cleaning_dicts.pos_dict).fillna(df['position'])

    
    df.insert(0, "p_id", (df['player']+'_'+df['team_name']+'_'+df['year']+'_'+df['week']))
    df.insert(1, "unique_team_id", (df['team_name']+'_'+df['year']+'_'+df['week']))
    df.insert(2, "player_team_id", (df['player']+'_'+df['team_name']+'_'+df['year']))
    df.insert(3, "team_id_impute", (df['team_name']+'_'+df['year']))
    return df

def_summ_conc = drop_non_def(def_summ_conc)
pass_rush_summary = drop_non_def(pass_rush_summary)
run_defense_summary = drop_non_def(run_defense_summary)
defense_coverage_scheme = drop_non_def(defense_coverage_scheme)
defense_coverage_summary = drop_non_def(defense_coverage_summary)
slot_coverage = drop_non_def(slot_coverage)	
	

####################################################################################
				###   Read-in and clean all special teams datasets ###
####################################################################################	

st_kickers = pd.read_csv('./historic_data/pff_data/st_kickers_hist.csv')
st_punters = pd.read_csv('./historic_data/pff_data/st_punters_hist.csv')

def clean_spec(df):
    df=df.rename(columns={"player_id": "numeric_id"})
    df['position']=df['position'].astype(str).str.lower()
    df['team_name']=df['team_name'].astype(str).str.lower()       
    df['player']=df['player'].str.replace('[^a-zA-Z0-9]', '').str.lower()
    df['team_name']=df['team_name'].str.lower()
    df['year'] = df['year'].astype(str)
    df['week'] = df['week'].astype(str)
        ##  pass team name through dictionary to clean ##
    df['team_name'] = df['team_name'].map(utils.cleaning_dicts.clean_team_pff).fillna(df['team_name'])
    df['position'] = df['position'].map(utils.cleaning_dicts.pos_dict).fillna(df['position'])

    
    df.insert(0, "p_id", (df['player']+'_'+df['team_name']+'_'+df['year']+'_'+df['week']))
    df.insert(1, "unique_team_id", (df['team_name']+'_'+df['year']+'_'+df['week']))
    df.insert(2, "player_team_id", (df['player']+'_'+df['team_name']+'_'+df['year']))
    df.insert(3, "team_id_impute", (df['team_name']+'_'+df['year']))
    return df

st_kickers =clean_spec(st_kickers)
st_punters = clean_spec(st_punters)



####################################################################################
####################################################################################
####################################################################################

/home/tomb/anaconda3/envs/cloak/lib/python3.7/site-packages/ipykernel_launcher.py:18: FutureWarning: The default value of regex will change from True to False in a future version.
/home/tomb/anaconda3/envs/cloak/lib/python3.7/site-packages/ipykernel_launcher.py:58: FutureWarning: The default value of regex will change from True to False in a future version.
/home/tomb/anaconda3/envs/cloak/lib/python3.7/site-packages/ipykernel_launcher.py:91: FutureWarning: The default value of regex will change from True to False in a future version.
/home/tomb/anaconda3/envs/cloak/lib/python3.7/site-packages/ipykernel_launcher.py:125: FutureWarning: The default value of regex will change from True to False in a future version.
/home/tomb/anaconda3/envs/cloak/lib/python3.7/site-packages/ipykernel_launcher.py:163: FutureWarning: The default value of regex will change from True to False in a future version.
/home/tomb/anaconda3/envs/cloak/lib/python3.7/site-packages/ipykernel_launcher.py:197: FutureWarni

In [13]:
passing_summ_conc.head(n=5)

,p_id,unique_team_id,player_team_id,team_id_impute,player,numeric_id,position,team_name,player_game_count,accuracy_percent,...,scrambles,spikes,thrown_aways,touchdowns,turnover_worthy_plays,twp_rate,yards,ypa,week,year
0,ryantannehill_ten_2021_1,ten_2021_1,ryantannehill_ten_2021,ten_2021,ryantannehill,7014,qb,ten,1,68.1,...,1,0,0,1,1,1.6,298,6.1,1,2021
1,jalenhurts_phi_2021_1,phi_2021_1,jalenhurts_phi_2021,phi_2021,jalenhurts,40291,qb,phi,1,76.2,...,4,0,4,2,2,3.3,387,8.1,1,2021
2,tombrady_tb_2021_1,tb_2021_1,tombrady_tb_2021,tb_2021,tombrady,698,qb,tb,1,67.5,...,1,0,2,0,0,0.0,269,6.3,1,2021
3,mattryan_atl_2021_1,atl_2021_1,mattryan_atl_2021,atl_2021,mattryan,4317,qb,atl,1,69.2,...,1,1,1,4,0,0.0,283,6.7,1,2021
4,matthewstafford_lar_2021_1,lar_2021_1,matthewstafford_lar_2021,lar_2021,matthewstafford,4924,qb,lar,1,70.7,...,2,0,0,2,1,2.1,280,6.8,1,2021


### Add prefixes to all columns.  Creating column names structured as "source-dataset_column-name"

In [14]:
####################################################################################
								###   add prefixes ###
####################################################################################	

def create_prefix(prefix=None, df=None):
    id = df[['p_id']]
    temp = df.drop(['p_id','player','numeric_id','position','team_name'], axis=1)
    temp = temp.add_prefix(prefix)
    id = pd.concat([id, temp], axis=1)
    return id

def id_prefix(prefix=None, df=None):
    id = df[['p_id','player','player_team_id','team_id_impute','numeric_id','position','team_name','year','week']]
    temp = df.drop(['p_id','player','player_team_id','team_id_impute','numeric_id','position','team_name','year','week'], axis=1)
    temp = temp.add_prefix(prefix)
    id = pd.concat([id, temp], axis=1)
    return id

passing_summ_conc = id_prefix(prefix="pass_summary_", df=passing_summ_conc)
rush_summ_conc = id_prefix(prefix="rush_summary_", df=rush_summ_conc)
rec_summ_conc = id_prefix(prefix="rec_summary_", df=rec_summ_conc)
block_summ_conc = id_prefix(prefix="block_summary_", df=block_summ_conc)
def_summ_conc = id_prefix(prefix="def_summary_", df=def_summ_conc)
st_kickers = id_prefix(prefix="kicking_", df=st_kickers)
st_punters = id_prefix(prefix="punting_", df=st_punters)


passing_depth = create_prefix(prefix="pass_depth_", df=passing_depth)
passing_allowed_pressure = create_prefix(prefix="pressure_source_", df=passing_allowed_pressure)
passing_pressure = create_prefix(prefix="pass_under_pressure_", df=passing_pressure)
passing_concept = create_prefix(prefix="pass_concept_", df=passing_concept)
time_in_pocket = create_prefix(prefix="pass_time_", df=time_in_pocket)


receiving_concept = create_prefix(prefix="rec_concept_", df=receiving_concept)
receiving_depth = create_prefix(prefix="rec_depth_", df=receiving_depth)
receiving_scheme = create_prefix(prefix="rec_scheme_", df=receiving_scheme)

offense_pass_blocking = create_prefix(prefix="pass_block_", df=offense_pass_blocking)
offense_run_blocking = create_prefix(prefix="run_block_", df=offense_run_blocking)


pass_rush_summary = create_prefix(prefix="pass_rush_", df=pass_rush_summary)
run_defense_summary = create_prefix(prefix="run_defense_", df=run_defense_summary)
defense_coverage_scheme = create_prefix(prefix="def_coverage_scheme_", df=defense_coverage_scheme)
defense_coverage_summary = create_prefix(prefix="def_coverage_summary_", df=defense_coverage_summary)
slot_coverage = create_prefix(prefix="def_slot_coverage_", df=slot_coverage)



### Read in weather data and clean raiders name - merged onto spreads data below ###

In [15]:
### read in weather data###
weather = pd.read_csv('/home/tomb/nfl_models/historic_data/weather_data/weather_hist.csv')

def raiders(df):
    if 'oak' in str(df.away_matchup_id) and '2020' in str(df.away_matchup_id):
        return df.away_matchup_id.replace("oak","lv")
    if 'oak' in str(df.away_matchup_id) and '2021' in str(df.away_matchup_id):
        return df.away_matchup_id.replace("oak","lv")
    if 'oak' in str(df.away_matchup_id) and '2022' in str(df.away_matchup_id):
        return df.away_matchup_id.replace("oak","lv")
    else:
        return df.away_matchup_id
weather['away_matchup_id'] = weather.apply(lambda df: raiders(df), axis=1)

### Create spreads data ###

In [16]:
####################################################################################
				###   spreads data cleaning and engineering ###
####################################################################################

spreads = pd.read_csv('./spreads_data/spreads.csv')

spreads = spreads[spreads['schedule_season']>=2014]
spreads = spreads[['schedule_season','schedule_week','team_home_abb','score_home','score_away','away_team_abb','team_favorite_id','spread_favorite','over_under_line','starting_spread', 'Total Score Open',
       'fav_team_open', 'fav_team_cur', 'remain_fav', 'spread_movement','ou_movement', 'strong_movement', 'fav_team_stronger']]
spreads['team_home_abb'] = spreads['team_home_abb'].astype(str)
spreads['team_favorite_id'] = spreads['team_favorite_id'].astype(str)
spreads['over_under_line'] = spreads['over_under_line'].astype(float)


def fav_spread(nData):
    if nData['team_home_abb'] == nData['team_favorite_id']:
        return nData['spread_favorite']
    elif nData['away_team_abb'] == nData['team_favorite_id']:
        return nData['spread_favorite']
    else:
        pass
spreads['fav_spread'] = spreads.apply(lambda nData: fav_spread(nData), axis=1)

def nonfav_spread(nData):
    if nData['team_home_abb'] != nData['team_favorite_id']:
        return nData['team_home_abb']
    elif nData['away_team_abb'] != nData['team_favorite_id']:
        return nData['away_team_abb']
    else:
        pass
spreads['team_notfav_id'] = spreads.apply(lambda nData: nonfav_spread(nData), axis=1)

def cover_or_not(nData):    
    if nData['team_home_abb'] == nData['team_favorite_id']:
        if ((nData['score_home']-nData['score_away']))+nData['spread_favorite'] > 0:
            return 'Cover'
        elif ((nData['score_home']-nData['score_away']))+nData['spread_favorite'] == 0:            
            return 'Push'       
        else:            
            return 'No Cover'
    elif nData['away_team_abb'] == nData['team_favorite_id']:        
        if ((nData['score_away']-nData['score_home']))+nData['spread_favorite'] > 0:            
            return 'Cover'        
        elif ((nData['score_away']-nData['score_home']))+nData['spread_favorite'] == 0:            
            return 'Push'        
        else:            
            return 'No Cover'
spreads['fav_cover'] = spreads.apply(lambda nData: cover_or_not(nData), axis=1)

def OU_or_not(nData):    
    if (nData['score_home']+nData['score_away']) > nData['over_under_line']:        
        return 'Over'    
    elif (nData['score_home']-nData['score_away']) == nData['over_under_line']:        
        return 'Push'    
    else:        
        return 'Under'
spreads['over_under_result'] = spreads.apply(lambda nData: OU_or_not(nData), axis=1)



spreads['schedule_season'] = spreads['schedule_season'].apply(int)    
spreads['schedule_week'] = spreads['schedule_week'].apply(int)  
data = spreads.sort_values(by=["team_home_abb","schedule_season","schedule_week"], ascending=[True, True, True])

def clean_spreads(df):
    ##  basic scrubbing to clean data ##    
    df['schedule_season'] = df['schedule_season'].apply(str)    
    df['schedule_week'] = df['schedule_week'].apply(str)        
    df=df.apply(lambda x: x.astype(str).str.lower())    
    #df['schedule_week']=df['schedule_week'].astype(str).str[:-2].astype(object)    
    #df['schedule_season'] = df['schedule_season'].astype(str).str[:-2].astype(object)        
    
    ##  create our unique ids  ##
    df.insert(0, "home_matchup_id", (df['team_home_abb']+'vs'+df['away_team_abb']+'_'+df['schedule_season']+'_'+df['schedule_week']))
    df.insert(1, "away_matchup_id", (df['away_team_abb']+'@'+df['team_home_abb']+'_'+df['schedule_season']+'_'+df['schedule_week']))
    df.insert(2, "home_id", (df['team_home_abb']+'_'+df['schedule_season']+'_'+df['schedule_week']))
    df.insert(3, "away_id", (df['away_team_abb']+'_'+df['schedule_season']+'_'+df['schedule_week']))
    return df
    
data = clean_spreads(data)

data = pd.merge(data, weather, on='away_matchup_id', how='left')


sh = data
sa = data

sh = sh.rename(columns={'home_id':'team_id'})
sh.drop('away_id', axis=1, inplace=True)

sa = sa.rename(columns={'away_id':'team_id'})
sa.drop('home_id', axis=1, inplace=True)

spread_comb = pd.concat([sh, sa], axis=0)
spread_comb['team_abb'] = spread_comb['team_id'].astype(str).str[:3]
spread_comb['team_abb'] = spread_comb['team_abb'].str.replace("_","")

def hora1(nData):
    if nData['team_favorite_id'] == nData['team_home_abb']:
        return 1
    elif nData['team_notfav_id'] == nData['team_home_abb']:
        return 1
    else:
        return 0
spread_comb['homeoraway'] = spread_comb.apply(lambda nData: hora1(nData), axis=1)

def hora(nData):
    if nData['team_favorite_id'] == nData['away_team_abb']:
        return 1
    else:
        return 0
spread_comb['fav_homeoraway'] = spread_comb.apply(lambda nData: hora(nData), axis=1)
#sh['fav_homeoraway'] = sh.apply(lambda nData: hora(nData), axis=1)

def ws(nData):
    if (nData['fav_homeoraway'] == 0) & (nData['fav_cover'] == 'cover'):
        return 1
    elif (nData['fav_homeoraway'] == 1) & (nData['fav_cover'] == 'no cover'):
        return 1
    else:
        return 0

def ls(nData):    
    if (nData['fav_homeoraway'] == 0) & (nData['fav_cover'] == 'no cover'):
        return 1
    elif (nData['fav_homeoraway'] == 1) & (nData['fav_cover'] == 'cover'):
        return 1
    else:
        return 0

spread_comb['ats_w'] = spread_comb.apply(lambda nData: ws(nData), axis=1)
spread_comb['ats_l'] = spread_comb.apply(lambda nData: ls(nData), axis=1)
spread_comb.head(n=5).T

,0,1,2,3,4
home_matchup_id,arivslac_2014_1,arivssf_2014_3,arivswas_2014_6,arivsphi_2014_8,arivslar_2014_10
away_matchup_id,lac@ari_2014_1,sf@ari_2014_3,was@ari_2014_6,phi@ari_2014_8,lar@ari_2014_10
team_id,ari_2014_1,ari_2014_3,ari_2014_6,ari_2014_8,ari_2014_10
schedule_season,2014,2014,2014,2014,2014
schedule_week,1,3,6,8,10
team_home_abb,ari,ari,ari,ari,ari
score_home,18,23,30,24,31
score_away,17,14,20,20,14
away_team_abb,lac,sf,was,phi,lar
team_favorite_id,ari,sf,ari,ari,ari


## Read in historic weekly football outsiders data.  Clean raiders name and computer rolling mean###

In [17]:
def raiders_fo(df):
    if 'oak' in str(df.team) and '2020' in str(df.year):
        return df.team_id.str.replace("oak_","lv_")
    elif 'oak' in str(df.team) and '2021' in str(df.year):
        return df.team_id.str.replace("oak_","lv_")
    else:
        return str(df.team_id)

def rolling_fo(data=None, roll_value=None, roll_type=None):
    
    """
        Args:
        data: input pandas dataframe to be rolled
        roll_value: input the number, default is three ## we will need to modify the function if we want more ##
        roll_type: 'mean','std', or 'var' are the only options at the point
        ## assign mean for a given team & year as opposed to the entire dataset
   
    """
    
    data = data.sort_values(by=["team","year","week"], ascending=[True, True, True])
    #data=data.fillna(data.mean())
    num_cols = ['total_dvoa', 'off_dvoa','off_pass_dvoa', 'off_rush_dvoa', 'def_dvoa', 'def_pass_dvoa','def_rush_dvoa', 'special_teams_dvoa']
    ids = data[['team_id', 'year', 'team', 'week', 'opp']].reset_index(drop=True)
   
    if roll_type == 'mean':
        roll3 = data.groupby(['team','year'])[num_cols].apply(lambda x : x.shift().rolling(roll_value).mean())
        roll2 = data.groupby(['team','year'])[num_cols].apply(lambda x : x.shift().rolling(roll_value-1).mean())
        roll1 = data.groupby(['team','year'])[num_cols].apply(lambda x : x.shift().rolling(roll_value-2).mean())
        roll3 = pd.DataFrame(roll3.combine_first(roll2).combine_first(roll1)).reset_index(drop=True)
        df = pd.concat([ids, roll3], axis=1)
    return df

In [18]:
fo = pd.read_csv('./historic_data/fo_data/fo_weekly_hist.csv')

fo['team_id'] = fo.apply(lambda df: raiders_fo(df), axis=1) 
fo_roll = rolling_fo(data=fo, roll_value=3, roll_type='mean')
fo_roll.head().T

,0,1,2,3,4
team_id,ari_2014_1,ari_2014_2,ari_2014_3,ari_2014_5,ari_2014_6
year,2014,2014,2014,2014,2014
team,ari,ari,ari,ari,ari
week,1,2,3,5,6
opp,sd,nyg,sf,den,was
total_dvoa,NaN,0.59,0.625,0.613333,0.56
off_dvoa,NaN,0.53,0.505,0.55,0.533333
off_pass_dvoa,NaN,0.53,0.45,0.516667,0.486667
off_rush_dvoa,NaN,0.51,0.57,0.55,0.56
def_dvoa,NaN,0.61,0.56,0.49,0.406667


### PFF team_game_summaries (tgs) clean and prepare for moving averages (if wanted)

In [21]:
tgs = pd.read_csv('./historic_data/pff_data/team_game_summaries_historic.csv')
tgs = tgs[tgs['year'] >= 2014]

tgs['team_name'] = tgs['team'].map(utils.cleaning_dicts.clean_team_pff_full).fillna(tgs['team'])
tgs['opponent_name'] = tgs['opponent'].map(utils.cleaning_dicts.clean_team_pff_opp).fillna(tgs['opponent'])

tgs['home_or_away']=tgs['home_or_away'].astype(str)

def home_team(nData):
    if str('@') in nData['home_or_away']:
        return nData['opponent_name']
    else:
        return nData['team_name']

tgs['home_team'] = tgs.apply(lambda nData: home_team(nData), axis=1)

def away_team(nData):
    if str('@') in nData['home_or_away']:
        return nData['team_name']
    else:
        return nData['opponent_name']
    
tgs['away_team'] = tgs.apply(lambda nData: away_team(nData), axis=1)

def clean_pff_team_summ(df):
##  basic scrubbing to clean data ##

    df['year'] = df['year'].astype(str)
    df['week'] = df['week'].astype(str)
    df['home_or_away']=np.where(df['home_or_away'] == "@", 1, 0)
    df['wl_int'] = np.where(df['wl'] == "W", 1, 0)
    df=df.replace('-','', regex=True)
    df=df.replace(' ','', regex=True)


    ##  create our unique ids  ##
    df.insert(1, "team_id_impute", (df['team_name']+'_'+df['year']))
    df.insert(2, "opponent_id", (df['opponent_name']+'_'+df['year']+'_'+df['week']))
    df.insert(3, "home_matchup_id", (df['home_team']+'vs'+df['away_team']+'_'+df['year']+'_'+df['week']))

    return df
   
tgs_clean = clean_pff_team_summ(tgs)
tgs_clean.head().T

,0,1,2,27,28
team,denverbroncos,tennesseetitans,philadelphiaeagles,seattleseahawks,newyorkgiants
team_id_impute,den_2020,ten_2020,phi_2021,sea_2014,nyg_2014
opponent_id,ne_2020_6,buf_2020_5,was_2021_15,was_2014_5,ind_2014_9
home_matchup_id,nevsden_2020_6,tenvsbuf_2020_5,phivswas_2021_15,wasvssea_2014_5,nygvsind_2014_9
year,2020,2020,2021,2014,2014
week,6,5,15,5,9
home_or_away,1,0,0,1,0
team_num,10,31,26,29,23
game_id,18601,18604,19821,3149,3207
opponent,NewEnglandPatriots,BuffaloBills,WashingtonFootballTeam,WashingtonRedskins,IndianapolisColts


### Lets create a simple PFF team_game_summary dataset for modeling with averages across all games for a full season

In [22]:
tgs_clean['game'] = 1

## subset columns to what we need ##
tgs_sub = tgs_clean[['game','team_name', 'opponent_name', 'home_team', 'away_team','year',
                     'week', 'wl_int', 'pf', 'pa', 'overall_performance','offense','pass','pass_blocking',
                     'run_blocking','receiving','rushing','defense','rush_defense','tackling','pass_rush',
                     'coverage','special_teams']]

## create a point differential variable ##
tgs_sub['point_diff']=tgs_sub.pf-tgs_sub.pa

tgs_sub['year']=tgs_sub['year'].astype(int)
tgs_sub['week']=tgs_sub['week'].astype(int)
## sort columns to prepare for rolling functions ##
tgs_sub = tgs_sub.sort_values(by=["team_name","year","week"], ascending=[True, True, True])


tgs_sub['wl_total'] = tgs_sub.groupby(by=['team_name', 'year'])['wl_int'].transform(lambda x: x.cumsum())
tgs_sub['game_count'] = tgs_sub.groupby(by=['team_name', 'year'])['game'].transform(lambda x: x.cumsum())

def cum_averages(var=None, new_name=None):
    test = tgs_sub.groupby(by=['team_name', 'year'])[var].expanding().mean().reset_index(0)
    test.reset_index(inplace=True)
    test.set_index('level_1', inplace=True)
    test = test[[var]]
    test.columns = [new_name]
    return pd.concat([tgs_sub, test], axis=1)
    
## calculate averages for pf (points_for), pa (points_Against), point_diff, and overall pff performance ###
## we can add all the pff columns from the TGS dataset but this is just an example ##
tgs_sub = cum_averages(var='pf', new_name='avg_pf')
tgs_sub = cum_averages(var='pa', new_name='avg_pa')
tgs_sub = cum_averages(var='point_diff', new_name='avg_point_diff')
tgs_sub = cum_averages(var='overall_performance', new_name='pff_ovr_total')
tgs_sub = cum_averages(var='offense', new_name='pff_offense_total')
tgs_sub = cum_averages(var='pass', new_name='pff_pass_total')
tgs_sub = cum_averages(var='pass_blocking', new_name='pff_pass_blocking_total')
tgs_sub = cum_averages(var='run_blocking', new_name='pff_run_blocking_total')
tgs_sub = cum_averages(var='receiving', new_name='pff_receiving_total')
tgs_sub = cum_averages(var='rushing', new_name='pff_rushing_total')
tgs_sub = cum_averages(var='defense', new_name='pff_defense_total')
tgs_sub = cum_averages(var='rush_defense', new_name='pff_rush_defense_total')
tgs_sub = cum_averages(var='tackling', new_name='pff_tackling_total')
tgs_sub = cum_averages(var='pass_rush', new_name='pff_pass_rush_total')
tgs_sub = cum_averages(var='coverage', new_name='pff_coverage_total')
tgs_sub = cum_averages(var='special_teams', new_name='pff_special_teams_total')

tgs_sub['wl_total']=tgs_sub['wl_total'].astype(int)
tgs_sub['game_count']=tgs_sub['game_count'].astype(int)
tgs_sub['win_percentage'] = (tgs_sub['wl_total']/tgs_sub['game_count'])

## create some ids so we van find the oppositions stats ##
tgs_sub['year']=tgs_sub['year'].astype(str)
tgs_sub['week']=tgs_sub['week'].astype(str)
tgs_sub['team_id']= tgs_sub.team_name.str.cat(tgs_sub.year, sep="_").str.cat(tgs_sub.week, sep="_")
tgs_sub['matchup_id']= tgs_sub.opponent_name.str.cat(tgs_sub.year, sep="_").str.cat(tgs_sub.week, sep="_")

## create an opponent dataframe where we will merge on their TGS stats ##
tgs_sub['year']=tgs_sub['year'].astype(int)
tgs_sub['week']=tgs_sub['week'].astype(int)

def id_prefix(prefix=None, df=None):
    id = df[['team_id','team_name', 'opponent_name', 'home_team', 'away_team', 'year','week','matchup_id']]
    temp = df.drop(['team_id','team_name', 'opponent_name', 'home_team', 'away_team', 'year','week','matchup_id'], axis=1)
    temp = temp.add_prefix(prefix)
    id = pd.concat([id, temp], axis=1)
    return id
    
opp_df = id_prefix(prefix="opp_", df=tgs_sub)

## subset opponent dataset and merge onto the tgs_Sub ##
opp_df = opp_df[['team_id', 'opp_wl_total', 'opp_game_count', 'opp_avg_pf', 'opp_avg_pa', 'opp_avg_point_diff',
                 'opp_pff_ovr_total', 'opp_win_percentage','opp_pff_offense_total','opp_pff_pass_total','opp_pff_pass_blocking_total',
                'opp_pff_run_blocking_total','opp_pff_receiving_total','opp_pff_rushing_total','opp_pff_defense_total','opp_pff_rush_defense_total',
                'opp_pff_tackling_total','opp_pff_pass_rush_total','opp_pff_coverage_total','opp_pff_special_teams_total']]

tgs_sub.wl_total = tgs_sub.wl_total.shift(1)
tgs_sub.game_count = tgs_sub.game_count.shift(1)
tgs_sub.avg_pf = tgs_sub.avg_pf.shift(1)
tgs_sub.avg_pa = tgs_sub.avg_pa.shift(1)
tgs_sub.avg_point_diff = tgs_sub.avg_point_diff.shift(1)
tgs_sub.pff_ovr_total = tgs_sub.pff_ovr_total.shift(1)

tgs_sub.pff_offense_total = tgs_sub.pff_offense_total.shift(1)
tgs_sub.pff_pass_total = tgs_sub.pff_pass_total.shift(1)
tgs_sub.pff_pass_blocking_total = tgs_sub.pff_pass_blocking_total.shift(1)
tgs_sub.pff_run_blocking_total = tgs_sub.pff_run_blocking_total.shift(1)
tgs_sub.pff_receiving_total = tgs_sub.pff_receiving_total.shift(1)
tgs_sub.pff_rushing_total = tgs_sub.pff_rushing_total.shift(1)
tgs_sub.pff_defense_total = tgs_sub.pff_defense_total.shift(1)
tgs_sub.pff_rush_defense_total = tgs_sub.pff_rush_defense_total.shift(1)
tgs_sub.pff_tackling_total = tgs_sub.pff_tackling_total.shift(1)
tgs_sub.pff_pass_rush_total = tgs_sub.pff_pass_rush_total.shift(1)
tgs_sub.pff_coverage_total = tgs_sub.pff_coverage_total.shift(1)
tgs_sub.pff_special_teams_total = tgs_sub.pff_special_teams_total.shift(1)
tgs_sub.win_percentage = tgs_sub.win_percentage.shift(1)


opp_df.opp_wl_total = opp_df.opp_wl_total.shift(1)
opp_df.opp_game_count = opp_df.opp_game_count.shift(1)
opp_df.opp_avg_pf = opp_df.opp_avg_pf.shift(1)
opp_df.opp_avg_pa = opp_df.opp_avg_pa.shift(1)
opp_df.opp_avg_point_diff = opp_df.opp_avg_point_diff.shift(1)
opp_df.opp_pff_ovr_total = opp_df.opp_pff_ovr_total.shift(1)

opp_df.opp_pff_offense_total = opp_df.opp_pff_offense_total.shift(1)
opp_df.opp_pff_pass_total = opp_df.opp_pff_pass_total.shift(1)
opp_df.opp_pff_pass_blocking_total = opp_df.opp_pff_pass_blocking_total.shift(1)
opp_df.opp_pff_run_blocking_total = opp_df.opp_pff_run_blocking_total.shift(1)
opp_df.opp_pff_receiving_total = opp_df.opp_pff_receiving_total.shift(1)
opp_df.opp_pff_rushing_total = opp_df.opp_pff_rushing_total.shift(1)
opp_df.opp_pff_defense_total = opp_df.opp_pff_defense_total.shift(1)
opp_df.opp_pff_rush_defense_total = opp_df.opp_pff_rush_defense_total.shift(1)
opp_df.opp_pff_tackling_total = opp_df.opp_pff_tackling_total.shift(1)
opp_df.opp_pff_pass_rush_total = opp_df.opp_pff_pass_rush_total.shift(1)
opp_df.opp_pff_coverage_total = opp_df.opp_pff_coverage_total.shift(1)
opp_df.opp_pff_special_teams_total = opp_df.opp_pff_special_teams_total.shift(1)
opp_df.opp_win_percentage = opp_df.opp_win_percentage.shift(1)

final = pd.merge(tgs_sub, opp_df, left_on='matchup_id', right_on='team_id', how='left')

final['year']=final['year'].astype(str)
final['week']=final['week'].astype(str)
final['home_matchup_id']= final.home_team.str.cat(final.away_team, sep="vs").str.cat(final.year, sep="_").str.cat(final.week, sep="_")
final.head()

/home/tomb/anaconda3/envs/cloak/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/home/tomb/anaconda3/envs/cloak/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/home/tomb/anaconda3/envs/cloak/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

,game,team_name,opponent_name,home_team,away_team,year,week,wl_int,pf,pa,...,opp_pff_run_blocking_total,opp_pff_receiving_total,opp_pff_rushing_total,opp_pff_defense_total,opp_pff_rush_defense_total,opp_pff_tackling_total,opp_pff_pass_rush_total,opp_pff_coverage_total,opp_pff_special_teams_total,home_matchup_id
0,1,ari,lac,ari,lac,2014,1,1,18,17,...,70.70,67.941176,66.194118,61.982353,55.505882,62.988235,63.417647,63.429412,70.176471,arivslac_2014_1
1,1,ari,nyg,nyg,ari,2014,2,1,25,14,...,53.30,56.300000,58.800000,59.300000,73.700000,62.900000,55.500000,49.100000,56.100000,nygvsari_2014_2
2,1,ari,sf,ari,sf,2014,3,1,23,14,...,72.75,71.400000,64.850000,64.450000,70.500000,69.450000,54.600000,68.700000,61.900000,arivssf_2014_3
3,1,ari,den,den,ari,2014,5,0,20,41,...,67.30,74.466667,62.400000,63.533333,68.133333,69.300000,59.866667,63.900000,62.133333,denvsari_2014_5
4,1,ari,was,ari,was,2014,6,1,30,20,...,62.14,69.240000,53.820000,52.980000,62.860000,58.720000,63.420000,46.320000,43.860000,arivswas_2014_6


In [23]:
final.columns

Index(['game', 'team_name', 'opponent_name', 'home_team', 'away_team', 'year',
       'week', 'wl_int', 'pf', 'pa', 'overall_performance', 'offense', 'pass',
       'pass_blocking', 'run_blocking', 'receiving', 'rushing', 'defense',
       'rush_defense', 'tackling', 'pass_rush', 'coverage', 'special_teams',
       'point_diff', 'wl_total', 'game_count', 'avg_pf', 'avg_pa',
       'avg_point_diff', 'pff_ovr_total', 'pff_offense_total',
       'pff_pass_total', 'pff_pass_blocking_total', 'pff_run_blocking_total',
       'pff_receiving_total', 'pff_rushing_total', 'pff_defense_total',
       'pff_rush_defense_total', 'pff_tackling_total', 'pff_pass_rush_total',
       'pff_coverage_total', 'pff_special_teams_total', 'win_percentage',
       'team_id_x', 'matchup_id', 'team_id_y', 'opp_wl_total',
       'opp_game_count', 'opp_avg_pf', 'opp_avg_pa', 'opp_avg_point_diff',
       'opp_pff_ovr_total', 'opp_win_percentage', 'opp_pff_offense_total',
       'opp_pff_pass_total', 'opp_pff_pass_bl

### Lets create a simple PFF team_game_summary dataset for modeling with averages across all games for a full season.

### Now lets grab the columns we need from the final TGS dataframe and add the spreads data ###

In [37]:
tgs_model = final[['home_matchup_id','home_team', 'away_team', 'year',
       'week', 'wl_total', 'game_count', 'avg_pf', 'avg_pa', 'avg_point_diff',
       'pff_ovr_total', 'pff_offense_total', 'pff_pass_total',
       'pff_pass_blocking_total', 'pff_run_blocking_total', 'pff_receiving_total', 'pff_rushing_total', 'pff_defense_total',
       'pff_rush_defense_total', 'pff_tackling_total', 'pff_pass_rush_total', 'pff_coverage_total', 'pff_special_teams_total','win_percentage','opp_wl_total', 'opp_game_count', 'opp_avg_pf',
       'opp_avg_pa', 'opp_avg_point_diff', 'opp_pff_ovr_total', 'opp_pff_offense_total',
       'opp_pff_pass_total', 'opp_pff_pass_blocking_total',
       'opp_pff_run_blocking_total', 'opp_pff_receiving_total',
       'opp_pff_rushing_total', 'opp_pff_defense_total',
       'opp_pff_rush_defense_total', 'opp_pff_tackling_total',
       'opp_pff_pass_rush_total', 'opp_pff_coverage_total',
       'opp_pff_special_teams_total',
       'opp_win_percentage']]

tgs_model['week']=tgs_model['week'].astype(int)
tgs_model= tgs_model[tgs_model['week'] >= 3]

spreads_for_tgs = spread_comb[['home_matchup_id','fav_homeoraway','team_favorite_id','fav_cover','spread_favorite',
       'over_under_line', 'starting_spread', 'Total Score Open', 'spread_movement','precip', 'dome', 'temperature', 'wind_mph',
       'ou_movement', 'strong_movement', 'fav_team_stronger', 'fav_spread','ats_w','ats_l']]

tgs_model = pd.merge(tgs_model, spreads_for_tgs, on='home_matchup_id', how='left')
tgs_model.drop_duplicates(['home_team','year','week'], inplace=True)
tgs_model.drop_duplicates(['home_matchup_id'], inplace=True)


tgs_model['home_is_fav'] = np.where(tgs_model['home_team'] == (tgs_model['team_favorite_id']), 1, 0)
homefav = tgs_model[tgs_model['home_is_fav'] == 1]
awayfav = tgs_model[tgs_model['home_is_fav'] == 0]



def tgs_homefav_vars(df):
    df['avg_pf_matchup'] = df['avg_pf']/df['opp_avg_pf']
    df['avg_pa_matchup'] = df['avg_pa']/df['opp_avg_pa']
    df['avg_point_diff_matchup'] = df['avg_point_diff']/df['opp_avg_point_diff']
    df['pff_ovr_total_matchup'] = df['pff_ovr_total']/df['opp_pff_ovr_total']
    
    df['pff_offense_total_matchup'] = df['pff_offense_total']/df['opp_pff_offense_total']
    df['pff_pass_total_matchup'] = df['pff_pass_total']/df['opp_pff_pass_total']
    df['pff_pass_blocking_total_matchup'] = df['pff_pass_blocking_total']/df['opp_pff_pass_blocking_total']
    df['pff_run_blocking_total_matchup'] = df['pff_run_blocking_total']/df['opp_pff_run_blocking_total']
    df['pff_receiving_total_matchup'] = df['pff_receiving_total']/df['opp_pff_receiving_total']
    df['pff_rushing_total_matchup'] = df['pff_rushing_total']/df['opp_pff_rushing_total']
    df['pff_defense_total_matchup'] = df['pff_defense_total']/df['opp_pff_defense_total']
    df['pff_rush_defense_total_matchup'] = df['pff_rush_defense_total']/df['opp_pff_rush_defense_total']
    df['pff_tackling_total_matchup'] = df['pff_tackling_total']/df['opp_pff_tackling_total']
    df['pff_pass_rush_total_matchup'] = df['pff_pass_rush_total']/df['opp_pff_pass_rush_total']
    df['pff_coverage_total_matchup'] = df['pff_coverage_total']/df['opp_pff_coverage_total']
    df['pff_special_teams_total_matchup'] = df['pff_special_teams_total']/df['opp_pff_special_teams_total']
    df['avg_pfbypa_matchup'] = df['avg_pf']/df['opp_avg_pa']
    df['avg_pabypf_matchup'] = df['avg_pa']/df['opp_avg_pf']
    df['pff_offvsdef_total_matchup'] = df['pff_offense_total']/df['opp_pff_defense_total']
    df['pff_defvsoff_total_matchup'] = df['pff_defense_total']/df['opp_pff_offense_total']
    df['pff_pass_vs_coverage_matchup'] = df['pff_pass_total']/df['opp_pff_coverage_total']
    df['pff_passblock_vs_passrush_total_matchup'] = df['pff_pass_blocking_total']/df['opp_pff_pass_rush_total']
    df['pff_runblock_vs_rushdef_total_matchup'] = df['pff_run_blocking_total']/df['opp_pff_rush_defense_total']
    df['pff_rec_vs_cov_total_matchup'] = df['pff_receiving_total']/df['opp_pff_coverage_total']
    df['pff_rush_vs_rushdef_total_matchup'] = df['pff_rushing_total']/df['opp_pff_rush_defense_total']
    df['pff_rush_vs_tack_total_matchup'] = df['pff_rushing_total']/df['opp_pff_tackling_total']
    df['pff_rushdef_vs_rushoff_total_matchup'] = df['pff_rush_defense_total']/df['opp_pff_rushing_total']
    df['pff_passrush_vs_passblock_total_matchup'] = df['pff_pass_rush_total']/df['opp_pff_pass_blocking_total']
    df['pff_coverage_vs_receiving_total_matchup'] = df['pff_coverage_total']/df['opp_pff_receiving_total']
    df['pff_coverage_vs_passblock_total_matchup'] = df['pff_coverage_total']/df['opp_pff_pass_total']
    
    df['win_percentage_matchup'] = df['win_percentage']/df['opp_win_percentage']
    df['wl_matchup'] = df['wl_total']/df['opp_wl_total']
    df['game_count_matchup'] = df['game_count']/df['opp_game_count']
    return df
    
def tgs_awayfav_vars(df):
    df['avg_pf_matchup'] = df['opp_avg_pf']/df['avg_pf']
    df['avg_pa_matchup'] = df['opp_avg_pa']/df['avg_pa']
    df['avg_point_diff_matchup'] = df['opp_avg_point_diff']/df['avg_point_diff']
    df['pff_ovr_total_matchup'] = df['opp_pff_ovr_total']/df['pff_ovr_total']
    
    df['pff_offense_total_matchup'] = df['opp_pff_offense_total']/df['pff_offense_total']
    df['pff_pass_total_matchup'] = df['opp_pff_pass_total']/df['pff_pass_total']
    df['pff_pass_blocking_total_matchup'] = df['opp_pff_pass_blocking_total']/df['pff_pass_blocking_total']
    df['pff_run_blocking_total_matchup'] = df['opp_pff_run_blocking_total']/df['pff_run_blocking_total']
    df['pff_receiving_total_matchup'] = df['opp_pff_receiving_total']/df['pff_receiving_total']
    df['pff_rushing_total_matchup'] = df['opp_pff_rushing_total']/df['pff_rushing_total']
    df['pff_defense_total_matchup'] = df['opp_pff_defense_total']/df['pff_defense_total']
    df['pff_rush_defense_total_matchup'] = df['opp_pff_rush_defense_total']/df['pff_rush_defense_total']
    df['pff_tackling_total_matchup'] = df['opp_pff_tackling_total']/df['pff_tackling_total']
    df['pff_pass_rush_total_matchup'] = df['opp_pff_pass_rush_total']/df['pff_pass_rush_total']
    df['pff_coverage_total_matchup'] = df['opp_pff_coverage_total']/df['pff_coverage_total']
    df['pff_special_teams_total_matchup'] = df['opp_pff_special_teams_total']/df['pff_special_teams_total']
    df['avg_pfbypa_matchup'] = df['opp_avg_pf']/df['avg_pa']
    df['avg_pabypf_matchup'] = df['opp_avg_pa']/df['avg_pf']
    df['pff_offvsdef_total_matchup'] = df['opp_pff_offense_total']/df['pff_defense_total']
    df['pff_defvsoff_total_matchup'] = df['opp_pff_defense_total']/df['pff_offense_total']
    df['pff_pass_vs_coverage_matchup'] = df['opp_pff_pass_total']/df['pff_coverage_total']
    df['pff_passblock_vs_passrush_total_matchup'] = df['opp_pff_pass_blocking_total']/df['pff_pass_rush_total']
    df['pff_runblock_vs_rushdef_total_matchup'] = df['opp_pff_run_blocking_total']/df['pff_rush_defense_total']
    df['pff_rec_vs_cov_total_matchup'] = df['opp_pff_receiving_total']/df['pff_coverage_total']
    df['pff_rush_vs_rushdef_total_matchup'] = df['opp_pff_rushing_total']/df['pff_rush_defense_total']
    df['pff_rush_vs_tack_total_matchup'] = df['opp_pff_rushing_total']/df['pff_tackling_total']
    df['pff_rushdef_vs_rushoff_total_matchup'] = df['opp_pff_rush_defense_total']/df['pff_rushing_total']
    df['pff_passrush_vs_passblock_total_matchup'] = df['opp_pff_pass_rush_total']/df['pff_pass_blocking_total']
    df['pff_coverage_vs_receiving_total_matchup'] = df['opp_pff_coverage_total']/df['pff_receiving_total']
    df['pff_coverage_vs_passblock_total_matchup'] = df['opp_pff_coverage_total']/df['pff_pass_total']
    
    df['win_percentage_matchup'] = df['opp_win_percentage']/df['win_percentage']
    df['wl_matchup'] = df['opp_wl_total']/df['wl_total']
    df['game_count_matchup'] = df['opp_game_count']/df['game_count']
    return df


homefav = tgs_homefav_vars(homefav)
awayfav = tgs_awayfav_vars(awayfav)


tgs_final = pd.concat([homefav, awayfav], axis=0)

/home/tomb/anaconda3/envs/cloak/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/home/tomb/anaconda3/envs/cloak/lib/python3.7/site-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/tomb/anaconda3/envs/cloak/lib/python3.7/site-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_in

In [38]:
tgs_final.head().T

,2,4,8,10,12
home_matchup_id,denvsari_2014_5,arivswas_2014_6,arivsphi_2014_8,dalvsari_2014_9,arivslar_2014_10
home_team,den,ari,ari,dal,ari
away_team,ari,was,phi,ari,lar
year,2014,2014,2014,2014,2014
week,5,6,8,9,10
...,...,...,...,...,...
pff_coverage_vs_receiving_total_matchup,0.701432,0.807698,0.905155,0.848912,0.929399
pff_coverage_vs_passblock_total_matchup,0.72312,0.819054,0.940546,0.848323,1.029951
win_percentage_matchup,1.5,3.75,1.0,1.142857,2.333333
wl_matchup,1.5,3.0,1.0,1.0,2.333333


In [39]:
tgs_final.columns

Index(['home_matchup_id', 'home_team', 'away_team', 'year', 'week', 'wl_total',
       'game_count', 'avg_pf', 'avg_pa', 'avg_point_diff', 'pff_ovr_total',
       'pff_offense_total', 'pff_pass_total', 'pff_pass_blocking_total',
       'pff_run_blocking_total', 'pff_receiving_total', 'pff_rushing_total',
       'pff_defense_total', 'pff_rush_defense_total', 'pff_tackling_total',
       'pff_pass_rush_total', 'pff_coverage_total', 'pff_special_teams_total',
       'win_percentage', 'opp_wl_total', 'opp_game_count', 'opp_avg_pf',
       'opp_avg_pa', 'opp_avg_point_diff', 'opp_pff_ovr_total',
       'opp_pff_offense_total', 'opp_pff_pass_total',
       'opp_pff_pass_blocking_total', 'opp_pff_run_blocking_total',
       'opp_pff_receiving_total', 'opp_pff_rushing_total',
       'opp_pff_defense_total', 'opp_pff_rush_defense_total',
       'opp_pff_tackling_total', 'opp_pff_pass_rush_total',
       'opp_pff_coverage_total', 'opp_pff_special_teams_total',
       'opp_win_percentage', 'fav_h

In [40]:
tgs_final = tgs_final[['home_matchup_id','team_favorite_id','fav_cover','year', 'week',
                      'spread_favorite', 'over_under_line', 'starting_spread',
       'Total Score Open', 'spread_movement', 'ou_movement', 'strong_movement',
       'fav_team_stronger', 'fav_spread', 'precip', 'dome', 'temperature','wind_mph', 'avg_pf_matchup', 'avg_pa_matchup',
       'avg_point_diff_matchup', 'pff_ovr_total_matchup',
       'pff_offense_total_matchup', 'pff_pass_total_matchup',
       'pff_pass_blocking_total_matchup', 'pff_run_blocking_total_matchup',
       'pff_receiving_total_matchup', 'pff_rushing_total_matchup',
       'pff_defense_total_matchup', 'pff_rush_defense_total_matchup',
       'pff_tackling_total_matchup', 'pff_pass_rush_total_matchup',
       'pff_coverage_total_matchup', 'pff_special_teams_total_matchup',
       'avg_pfbypa_matchup', 'avg_pabypf_matchup',
       'pff_offvsdef_total_matchup', 'pff_defvsoff_total_matchup',
       'pff_pass_vs_coverage_matchup',
       'pff_passblock_vs_passrush_total_matchup',
       'pff_runblock_vs_rushdef_total_matchup', 'pff_rec_vs_cov_total_matchup',
       'pff_rush_vs_rushdef_total_matchup', 'pff_rush_vs_tack_total_matchup',
       'pff_rushdef_vs_rushoff_total_matchup',
       'pff_passrush_vs_passblock_total_matchup',
       'pff_coverage_vs_receiving_total_matchup',
       'pff_coverage_vs_passblock_total_matchup',
       'win_percentage_matchup', 'wl_matchup', 'game_count_matchup','ats_w','ats_l']]

tgs_final.replace([np.inf, -np.inf], 0, inplace=True)


In [41]:
tgs_model.tail(n=10).T

,6448,6490,6522,6542,6562,6654,6678,6738,6764,6892
home_matchup_id,wasvssf_2019_7,sfvswas_2020_14,tenvssf_2021_16,wasvstb_2014_11,wasvstb_2015_7,tbvswas_2018_10,tenvstb_2019_8,wasvstb_2021_10,wasvsten_2014_7,tenvswas_2018_16
home_team,was,sf,ten,was,was,tb,ten,was,was,ten
away_team,sf,was,sf,tb,tb,was,tb,tb,ten,was
year,2019,2020,2021,2014,2015,2018,2019,2021,2014,2018
week,7,14,16,11,7,10,8,10,7,16
...,...,...,...,...,...,...,...,...,...,...
fav_team_stronger,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0
fav_spread,-10.0,-3.0,-3.0,-6.5,-3.0,-3.5,-2.0,-10.0,-6.0,-11.5
ats_w,1,0,1,0,0,0,1,1,0,0
ats_l,0,1,0,1,1,1,0,0,1,1


In [42]:
tgs_model.drop_duplicates('home_matchup_id', inplace=True)
tgs_model.to_csv("~/nfl_models/modeling_data/tgs_models.csv", index=False)

In [ ]:
passing_summ_conc.head()

In [ ]:
##  clean the player name field from pfr function above
passing_summ_conc=passing_summ_conc.fillna(0)
passing_summ_conc.drop_duplicates(['player','team_name','year','week'], inplace=True)


def rolling(data=None, roll_value=None, roll_type=None):
    """
        Args:
        data: input pandas dataframe to be rolled
        roll_value: input the number, default is three ## we will need to modify the function if we want more ##
        roll_type: 'mean','std', or 'var' are the only options at the point
        ## assign mean for a given team & year as opposed to the entire dataset
    """
    data['year']=data['year'].astype(int)
    data['week']=data['week'].astype(int)
    data = data.sort_values(by=["player","team_name","year","week"], ascending=[True, True, True, True])

    num_cols = data.select_dtypes(include=[np.number]).columns.tolist()
    ids = pd.DataFrame(data.select_dtypes(exclude=[np.number])).reset_index(drop=True)
   
    if roll_type == 'mean':
        #roll5 = data.groupby(data['player_id'])[num_cols].apply(lambda x : x.shift().rolling(roll_value).mean())
        #roll4 = data.groupby(data['player_id'])[num_cols].apply(lambda x : x.shift().rolling(roll_value).mean())
        roll3 = data.groupby(data['player_team_id'])[num_cols].apply(lambda x : x.shift().rolling(roll_value).mean())
        roll2 = data.groupby(data['player_team_id'])[num_cols].apply(lambda x : x.shift().rolling(roll_value-1).mean())
        roll1 = data.groupby(data['player_team_id'])[num_cols].apply(lambda x : x.shift().rolling(roll_value-2).mean())
        roll3 = pd.DataFrame(roll3.combine_first(roll2).combine_first(roll1)).reset_index(drop=True)
        df = pd.concat([ids, roll3], axis=1)
    elif roll_type == 'std':
        #roll5 = data.groupby(data['player_id'])[num_cols].apply(lambda x : x.shift().rolling(roll_value).std())
        #roll4 = data.groupby(data['player_id'])[num_cols].apply(lambda x : x.shift().rolling(roll_value-1).std())
        roll3 = data.groupby(data['player_id'])[num_cols].apply(lambda x : x.shift().rolling(roll_value).std())
        roll2 = data.groupby(data['player_id'])[num_cols].apply(lambda x : x.shift().rolling(roll_value-2).std())
        roll1 = data.groupby(data['player_id'])[num_cols].apply(lambda x : x.shift().rolling(roll_value-3).std())
        roll3 = pd.DataFrame(roll3.combine_first(roll2).combine_first(roll1)).reset_index(drop=True)
        df = pd.concat([ids, roll3], axis=1)
    return df
   
passing_summ_conc = rolling(data=passing_summ_conc, roll_value=3, roll_type='mean')


In [ ]:
passing_summ_conc.head()
#roll_rbs= roll_rbs[~roll_rbs['year'].isin(['nan'])]

In [ ]:
      
profiles = pd.read_csv('./other_data/rt_imputed.csv')
profiles=profiles[profiles['positionclean']=='qb']
qb_df = profiles[['unique_id','pff_name','positionclean','height_clean','weight_clean','speed_clean',
                  'hand_size','arm_length', 'bench','vertical','broad_jump','shuttle','3cone','explosive',
                  'size_speed','draft_yr','round','selection']]


qb_df.columns = [str(col) + '_qbs' for col in qb_df.columns]
passing_summ_conc = pd.merge(passing_summ_conc, qb_df, left_on='player_team_id', right_on='unique_id_qbs', how='left')
passing_summ_conc.drop_duplicates(subset=['p_id'], keep='first', inplace=True)
passing_summ_conc.head()

In [ ]:
passing_summ_conc.columns
roll_qbs=passing_summ_conc

In [ ]:

def pass_att(nData):
    if nData['pass_summary_dropbacks'] == 0:
        return 1
    else:
        return nData['pass_summary_dropbacks']
roll_qbs['pass_summary_dropbacks'] = roll_qbs.apply(lambda df: pass_att(df), axis=1) 

def game_combine(data=None):
    data_cols = data.select_dtypes(include=[np.number])
    num_cols = data_cols[data_cols.columns.drop(list(data_cols.filter(regex=r'grades_|week|year|team_id|height|wt|speed|draft_yr|round|selection|franchise|explosive_|vertical|3cone|shuttle|broad|bench|arm_|hand_size')))].columns.tolist()
    group = data.groupby(data['pass_summary_unique_team_id'])[num_cols].mean().reset_index()
    return group

def weighted(nData, snap_Var='pass_summary_dropbacks'):
    pff_cols = nData.filter(like='grades_').columns
    return pd.Series(np.average(nData[pff_cols], weights=nData[snap_Var], axis=0), pff_cols)

def pros(nData, snap_Var='pass_summary_dropbacks'):
    pro_cols = nData.filter(regex='height|wt|speed|draft_yr|round|selection|explosive_|vertical|3cone|shuttle|broad|bench|arm_|hand_size').columns
    return pd.Series(np.average(nData[pro_cols], weights=nData[snap_Var], axis=0), pro_cols)

roll_qbs_avgs=roll_qbs.fillna(roll_qbs.mean())
qb_pro_avgs = roll_qbs_avgs.groupby('pass_summary_unique_team_id').apply(pros).reset_index()

roll_qbs_zeros=roll_qbs.fillna(0.1)
qb_pff_avgs = roll_qbs_zeros.groupby('pass_summary_unique_team_id').apply(weighted).reset_index()

qb_avgs = game_combine(data=roll_qbs)

qb_stats = pd.merge(qb_pff_avgs, qb_pro_avgs, on='pass_summary_unique_team_id', how='left').merge(qb_avgs, on='pass_summary_unique_team_id', how='left')
qb_stats.tail().T                     

#Create opp dataset

In [ ]:
def id_prefix(prefix=None, df=None):
    id = df[['team_id','team_name', 'opponent_name', 'home_team', 'away_team', 'year','week','matchup_id']]
    temp = df.drop(['team_id','team_name', 'opponent_name', 'home_team', 'away_team', 'year','week','matchup_id'], axis=1)
    temp = temp.add_prefix(prefix)
    id = pd.concat([id, temp], axis=1)
    return id
    
opp_df = id_prefix(prefix="opp_", df=tgs_sub)

## subset opponent dataset and merge onto the tgs_Sub ##
opp_df = opp_df[['team_id', 'opp_wl_total', 'opp_game_count', 'opp_avg_pf', 'opp_avg_pa', 'opp_avg_point_diff',
                 'opp_pff_ovr_total', 'opp_win_percentage','opp_pff_offense_total','opp_pff_pass_total','opp_pff_pass_blocking_total',
                'opp_pff_run_blocking_total','opp_pff_receiving_total','opp_pff_rushing_total','opp_pff_defense_total','opp_pff_rush_defense_total',
                'opp_pff_tackling_total','opp_pff_pass_rush_total','opp_pff_coverage_total','opp_pff_special_teams_total']]


In [ ]:
spreads_for_qb_summ = spread_comb[['home_matchup_id','away_matchup_id','team_id','fav_homeoraway','team_favorite_id','fav_cover','spread_favorite',
       'over_under_line', 'starting_spread', 'Total Score Open', 'spread_movement',
       'ou_movement', 'strong_movement', 'fav_team_stronger', 'fav_spread','precip', 'dome',
       'temperature', 'wind_mph','ats_w','ats_l']]


In [ ]:
spreads_for_qb_summ.tail(n=10)

In [ ]:
tgs_clean.columns

In [ ]:
tgs_clean['team_id'] = tgs_clean['team_id_impute']+'_'+tgs_clean['week']

In [ ]:
tgs_clean.head().T

In [ ]:
tgs_clean_pass_summ = tgs_clean[['team_id','opponent_id','home_matchup_id']]

In [ ]:
qb_stats = pd.merge(qb_stats, tgs_clean_pass_summ, left_on='pass_summary_unique_team_id', right_on='team_id', how='left')
qb_stats = pd.merge(qb_stats, spreads_for_qb_summ, on='home_matchup_id', how='left')
qb_stats.head()

In [ ]:
qb_stats.columns